In [1]:
import random
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
ds = load_dataset("bdsaglam/musique", "answerable")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
        num_rows: 19938
    })
    validation: Dataset({
        features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable'],
        num_rows: 2417
    })
})

In [3]:
ds = ds.map(lambda x: {'n_hops': len(x['question_decomposition'])})
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable', 'n_hops'],
        num_rows: 19938
    })
    validation: Dataset({
        features: ['id', 'paragraphs', 'question', 'question_decomposition', 'answer', 'answer_aliases', 'answerable', 'n_hops'],
        num_rows: 2417
    })
})

In [4]:
pd.Series(ds['train']['n_hops']).value_counts()

2    14376
3     4387
4     1175
Name: count, dtype: int64

In [5]:
pd.Series(ds['validation']['n_hops']).value_counts()

2    1252
3     760
4     405
Name: count, dtype: int64

In [6]:
# filter in examples with 4 hops
mds = ds.filter(lambda x: x['n_hops'] == 4)

In [7]:
def print_example(example):
    print(f"Question: {example['question']}")
    for i, d in enumerate(example['question_decomposition']):
        subq = d['question']
        print(f"  {i+1}. {subq}")

In [8]:
# choose 5 random examples and print them. be aware mds is a HF dataset

for i in range(5):
    idx = random.randint(0, len(mds['train']) - 1)
    print_example(mds['train'][idx])
    print()

Question: What did Goring believe the operator of battleships from the country of the military that follows the Reichswehr would gain with further support?
  1. Reichswehr >> follows
  2. #1 >> country
  3. list of battleships of #2 >> operator
  4. What did Goring believe the #3 would gain with further support?

Question: Of the countries in the São Lourenço River's continent, what is the most popular sport in the one that provided the most legal immigrants in 2013?
  1. São Lourenço River >> mouth of the watercourse
  2. Which continent is #1 on?
  3. Of all the countries in #2 , which provided the most legal immigrants in 2013?
  4. what is the most popular sport in #3

Question: What is the total area of the capitol of the state where Georges Doriot was educated?
  1. Georges Doriot >> educated at
  2. #1 >> located in the administrative territorial entity
  3. What is the capital of #2 ?
  4. What is the total area of #3 ?

Question: Who was the mother of the ruler, under whom the

In [11]:
# a function that samples from the dataset with equal distribution of n_hops
def sample_evenly(dataset, n_samples):
    n_hops = np.unique(dataset['n_hops'])
    samples_per_hop = n_samples // len(n_hops)
    for hop in n_hops:
        hop_samples = dataset.filter(lambda x: x['n_hops'] == hop).shuffle().select(range(samples_per_hop))
        yield from hop_samples

In [12]:
samples = list(sample_evenly(ds['train'], 10))

Filter:   0%|          | 0/19938 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19938 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19938 [00:00<?, ? examples/s]